### Imports

In [ ]:
import sys
import numpy as np
import pandas as pd
import os
from masserstein import NMRSpectrum, estimate_proportions
import matplotlib.pyplot as plt
import pulp
import pickle

### Data

In [ ]:
experiment_name = 'Saccharose hydrolysis'

In [ ]:
integrals_path = '../../mnova_integrals/sucrose_better_baseline_constant_region_sizes_calki.csv'

reagents_names = ['saccharose', 'alpha-glucose','beta-glucose', 'fructose']

integrals_separator = ','

output_path = '../../mnova_integrals/'

#### Mixture in time

In [ ]:
integrals = pd.read_csv(integrals_path, sep = integrals_separator, skiprows=2).iloc[:,[2,4,6,8]]
integrals.fillna(0., inplace=True)
integrals.columns = ['saccharose', 'alpha-glucose', 'beta-glucose', 'fructose']

In [ ]:
integrals[integrals < 0] = 0

In [ ]:
integrals

In [ ]:
for i in range(integrals.shape[1]):
    plt.plot(range(1,1024), integrals.iloc[:,i], '.')

### Saving preprocessed spectra

In [ ]:
integrals.to_csv(output_path + 'sacharoza_calki_nowe.csv', '\t', index=False)